# HW 2

\begin{equation*}
    \|x\|_1 = \max\limits_{i \in {1, ..., n}} |x_i|
\end{equation*}

\begin{equation*}
    \|A\|_1 = \max\limits_{i \in {1, ..., n}} \sum_{j = 1}^{n} |a_{ij}|
\end{equation*}

\begin{equation*}
    \|x\|_2 = \sum_{i = 1}^{n} |x_i|
\end{equation*}

\begin{equation*}
    \|A\|_2 = \max\limits_{j \in {1, ..., n}} \sum_{i = 1}^{n} |a_{ij}|
\end{equation*}

\begin{equation*}
    \|x\|_3 = \sqrt{(x, x)}
\end{equation*}

\begin{equation*}
    \|A\|_3 = \max\limits_{i \in {1, ..., n}} \sqrt{\lambda_i (A^{\ast} A)}
\end{equation*}

In [167]:
import numpy as np
from matplotlib import pyplot as plt

In [168]:
def norm_1 (obj):
    return np.linalg.norm(obj, ord = np.inf)

def norm_2 (obj):
    return np.linalg.norm(obj, ord = 1)

def norm_3 (obj):
    return np.linalg.norm(obj, ord = 2)

def direct_method_residual (A, b, method, norm):
    return norm(A @ method(A, b) - b)

def check_correctness (A, b):
    if A.shape[0] != A.shape[1]:
        raise RuntimeError("Solving systems of linear equation with non-square matrix"
                           "is not supported")
    if (A.shape[0] != b.shape[0]):
        raise RuntimeError("Sizes of matrix and vector mismatch")

class Norm:
    def __init__(self, formula, name):
        self.formula_ = formula
        self.name_ = name

## Gaussian elimination with pivoting

In [169]:
def swap_rows (A, b, i, j):

    A[[i, j]] = A[[j, i]]
    b[i], b[j] = b[j], b[i]

#Находим строку с максимальным по модулю элементом в столбце j, начиная со строки i
def find_pivot (A, i, j):

    pivot_row = i
    pivot = A[i, j]

    for row_i in range (i, A.shape[0]):
        elem = A[row_i][j]
        if (abs(pivot) < abs(elem)):
            pivot_row = row_i
            pivot = elem

    return pivot_row

#Решаем систему с верхнетреугольной матрицей A
def back_substitution (A, b):

    dim = A.shape[0]

    x = np.empty(dim, dtype=float)
    for i in range (dim - 1, -1, -1):
        x[i] = (b[i] - sum(A[i, k] * x[k] for k in range (i + 1, dim))) / A[i, i]

    return x

def Gaussian_elimination (A_, b_):

    check_correctness(A_, b_)

    A = A_.copy()
    b = b_.copy()
    dim = A.shape[0]

    for row_i in range (0, dim - 1):

        pivot_row = find_pivot(A, row_i, row_i)

        if (row_i != pivot_row):
            swap_rows (A, b, row_i, pivot_row)

        #Иключаем переменные
        for j in range (row_i + 1, dim):
            m = A[j, row_i] / A[row_i, row_i]
            A[j] -= m * A[row_i]
            b[j] -= m * b[row_i]

    return back_substitution(A, b)

## Input data

In [170]:
def load_matrix_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    data = []
    for line in lines:
        if line.strip() == '':
            continue
        row = [float(x) for x in line.split()]
        data.append(row)
    A = np.array(data, dtype=float)
    return A

A = load_matrix_from_file("111/bad_matrix7.txt")
B = load_matrix_from_file("111/good_matrix7.txt")
dimA = A.shape
dimB = B.shape
print("Matrix are loaded: dimA =" , dimA , "; dimB =" , dimB)
a = np.arange (1, dimA[0] + 1, dtype = float)
b = np.arange (1, dimB[0] + 1, dtype = float)
#

Matrix are loaded: dimA = (20, 20) ; dimB = (20, 20)


## Results

In [171]:
norms_list = [Norm(norm_1, "x1"),
              Norm(norm_2, "x2"),
              Norm(norm_3, "x3")]

print ("Gaussian elimination for matrix A (bad):")
for i in range (len (norms_list)):
    residual =  direct_method_residual(A, a, Gaussian_elimination, norms_list[i].formula_)
    print (f"- norm {i + 1}: {residual}")

print ("Gaussian elimination for matrix B (good):")
for i in range (len (norms_list)):
    residual = direct_method_residual (B, b, Gaussian_elimination, norms_list[i].formula_)
    print (f"- norm {i + 1}: {residual}")

Gaussian elimination for matrix A (bad):
- norm 1: 6.821210263296962e-13
- norm 2: 6.0254023992456496e-12
- norm 3: 1.5956708065935117e-12
Gaussian elimination for matrix B (good):
- norm 1: 5.329070518200751e-15
- norm 2: 2.7977620220553945e-14
- norm 3: 9.780050345373876e-15
